In [4]:
from google.colab import drive
drive.mount('/content/drive')
dataPath = "drive/UCFCourseWork/CAP4630ArtificailIntelligence"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##Unziping training and testing datasets

In [5]:
!7z x drive/'My Drive'/UCFCourseWork/'CAP4630ArtificialIntelligence '/RuTanks7000_v1.7z
!cd RuTanks7000_v1/test && ls
!rm -rf RuTanks7000_v1/train/'Civilian Car' && rm -rf RuTanks7000_v1/test/'Civilian Car'
!ls


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 818580040 bytes (781 MiB)

Extracting archive: drive/My Drive/UCFCourseWork/CAP4630ArtificialIntelligence /RuTanks7000_v1.7z
--
Path = drive/My Drive/UCFCourseWork/CAP4630ArtificialIntelligence /RuTanks7000_v1.7z
Type = 7z
Physical Size = 818580040
Headers Size = 435873
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 17         0% 182 - RuTanks7000_v1/train/Background/Background_1052.jpg                                                                0% 183 - RuTanks7000_v1/train/Background/Background_1053.jpg

## convert data from directory into data generator

In [0]:
import os
base_dir = 'RuTanks7000_v1'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')


In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

##Build the model with VGG16 Conv net

In [8]:
from keras.applications import VGG16
from keras import layers
from keras import models
from keras import optimizers

conv_base = VGG16(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3))

conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
# no dropout
model.add(layers.Dense(6, activation='sigmoid'))





58892288/58889256 [==============================] - 2s 0us/step








## Final tweeks to create data generator and compile model

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
batchSize = 32
# data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=batchSize,
    class_mode="categorical")

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=batchSize,
    class_mode="categorical")

# compile model

model.compile(
    loss='mean_squared_error', 
    optimizer=optimizers.RMSprop(lr=2e-5), 
    metrics=['acc'])

# train



Found 42000 images belonging to 6 classes.
Found 90 images belonging to 6 classes.



## Train Model

In [10]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)



Epoch 1/10
10/10 [==============================] - 268s 27s/step - loss: 0.1836 - acc: 0.1594 - val_loss: 0.1525 - val_acc: 0.1782
Epoch 2/10
10/10 [==============================] - 266s 27s/step - loss: 0.1422 - acc: 0.2344 - val_loss: 0.1446 - val_acc: 0.1889
Epoch 3/10
10/10 [==============================] - 267s 27s/step - loss: 0.1355 - acc: 0.2812 - val_loss: 0.1413 - val_acc: 0.1656
Epoch 4/10
10/10 [==============================] - 268s 27s/step - loss: 0.1334 - acc: 0.3281 - val_loss: 0.1417 - val_acc: 0.2447
Epoch 5/10
10/10 [==============================] - 265s 27s/step - loss: 0.1297 - acc: 0.3656 - val_loss: 0.1411 - val_acc: 0.2323
Epoch 6/10
10/10 [==============================] - 267s 27s/step - loss: 0.1270 - acc: 0.4031 - val_loss: 0.1429 - val_acc: 0.2557
Epoch 7/10
10/10 [==============================] - 266s 27s/step - loss: 0.1284 - acc: 0.3781 - val_loss: 0.1418 - val_acc: 0.2307
Epoch 8/10
10/10 [==============================] - 265s 27s/step - loss: 

## Finetuning

In [0]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [13]:
model.compile(
    loss='mean_squared_error',
    #
    # choose a smaller learning rate
    #
    optimizer=optimizers.RMSprop(lr=1e-5), 
    metrics=['acc'])

# train

history = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=10)

Epoch 1/2
10/10 [==============================] - 108s 11s/step - loss: 0.1188 - acc: 0.4656 - val_loss: 0.1418 - val_acc: 0.2980
Epoch 2/2
10/10 [==============================] - 105s 10s/step - loss: 0.1132 - acc: 0.4625 - val_loss: 0.1438 - val_acc: 0.2881
